In [1]:
from preamble import *
%matplotlib inline

ImportError: No module named preamble

## 5. Model Evaluation and Improvement

In [2]:
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# create a synthetic dataset
X, y = make_blobs(random_state=0)
print("X.shape:", X.shape)
print("y.shape:", y.shape)

# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# instantiate a model and fit it to the training set
logreg = LogisticRegression().fit(X_train, y_train)

# evaluate the model on the test set
print("Test set score: {:.2f}".format(logreg.score(X_test, y_test)))

('X.shape:', (100L, 2L))
('y.shape:', (100L,))
Test set score: 0.88


### 5.1 Cross-Validation
- 교차 검증
  - 데이터를 여러 번 반복해서 나누어 모델 학습
- K-Fold cross-vailidation
  - Fold: 원본 데이터에 대한 부분 집합
  - K로는 5나 10을 주로 사용
    - 첫번째 모델은 첫번째 fold를 테스트 데이터로 사용하고 나머지를 훈련 데이터로 사용
    - 두번째 모델은 두번째 fold를 테스트 데이터로 사용하고 나머지를 훈련 데이터로 사용
    - 세번째 모델은...

In [3]:
import matplotlib
print(matplotlib.__version__)

mglearn.plots.plot_cross_validation()

2.0.2


NameError: name 'mglearn' is not defined

#### 5.1.1 Cross-Validation in scikit-learn
- scikit-learn의 교차 검증
  - model_selection.cross_val_score(estimator, X, y=None, cv=None) 함수 사용
    - estimator
      - estimator object implementing ‘fit’
      - The object to use to fit the data.
    - X
      - The data to fit.
    - y
      - The target variable to try to predict in the case of supervised learning.
    - cv
      - K-Fold의 K값 (기본 값: 3)

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

iris = load_iris()
print("iris.data.shape:", iris.data.shape)
print("iris.target.shape:", iris.target.shape)

logreg = LogisticRegression()

scores = cross_val_score(logreg, iris.data, iris.target)
print("Cross-validation scores: {}".format(scores))

('iris.data.shape:', (150L, 4L))
('iris.target.shape:', (150L,))
Cross-validation scores: [ 0.96078431  0.92156863  0.95833333]


In [5]:
scores = cross_val_score(logreg, iris.data, iris.target, cv=5)
print("Cross-validation scores: {}".format(scores))

Cross-validation scores: [ 1.          0.96666667  0.93333333  0.9         1.        ]


- 교차 검증의 정확도: 각 교차 검증 정확도의 평균값 사용

In [6]:
print("Average cross-validation score: {:.2f}".format(scores.mean()))

Average cross-validation score: 0.96


### 5.1.2 Benefits of Cross-Validation
- 기존 train_test_split 방법만 사용하는 경우
  - 확보한 원본 데이터 중 일부의 데이터는 훈련 데이터로 활용하지 않으면서 모델을 구성함.
- cross_val_score 함수를 사용하는 경우
  - 데이터를 고르게 사용하여 fit을 하고 score를 구하기 때문에 모델의 성능을 좀 더 정확히 측정할 수 있음
  - 새로은 테스트 데이터의 예측 정확도에 대하여 최악과 최선의 경우를 짐작할 수 있음 
  - [주의] **cross_val_score가 직접 모델을 구성하는 방법은 아님!**
    - 즉, <u>이 함수를 호출하면 내부적으로 K번 모델을 구성하지만, 그러한 모델들은 평가의 목적으로만 활용됨.</u>

### 5.1.3 Stratified K-Fold cross-validation and other strategies
- 계층별 K-Fold 교차 검증
  - 각 Fold안의 클래스 비율이 전체 원본 데이터셋에 있는 클래스 비율과 동일하도록 맞춤
  - 즉, 원본 데이터셋에서 클래스 A가 90%, 클래스 B가 10% 비율이라면, 계층별 K-Fold 교차 검증에서 각 K개의 Fold안에는 클래스 A가 90%, 클래스 B가 10% 비율이 됨.
- scikit-learn의 cross_val_score 기본 설정
  - 분류모델: StratifiedKFold를 사용하여 기본적으로 계층별 K-Fold 교차 검증 수행
  - 회귀모델: 단순한 KFold를 사용하여 계층별이 아닌 기본 K-Fold 교차 검증 수행
    - 대신 회귀모델에서는 KFold를 사용할 때 shuffle 매개변수를 True로 지정하여 폴드를 나누기 전에 무작위로 데이터를 섞는 작업 추천

In [8]:
from sklearn.datasets import load_iris
iris = load_iris()
print("Iris labels:\n{}".format(iris.target))

Iris labels:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [9]:
mglearn.plots.plot_stratified_cross_validation()

NameError: name 'mglearn' is not defined

### More control over cross-validation
- 기본적으로...
  - 분류: StratifiedKFold가 사용됨
  - 회귀: KFold가 사용됨
- 하지만, 때때로 분류에 KFold가 사용되어야 할 필요도 있음
  - 다른 사람이 이미 수행한 사항을 재현해야 할 때
  - StratifiedKFold가 아닌 KFold를 생성하여 cross_val_score()의 cv 인자에 할당

In [10]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5) #교차 검증 분할기의 역할 수행

In [11]:
print("Cross-validation scores:\n{}".format(cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

Cross-validation scores:
[ 1.          0.93333333  0.43333333  0.96666667  0.43333333]


- 이런 경우 3-Fold를 사용하면 데이터 타겟 레이블 분포 특성상 성능이 매우 나쁠 수 있음

In [12]:
kfold = KFold(n_splits=3)
print("Cross-validation scores:\n{}".format(cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

Cross-validation scores:
[ 0.  0.  0.]


- 해결책
  - KFold를 만들 때 shuffle=True를 통해 데이터를 임의로 섞음.
  - random_state=0을 주면 추후 그대로 재현이 가능

In [13]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("Cross-validation scores:\n{}".format(cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

Cross-validation scores:
[ 0.9   0.96  0.96]


#### Leave-one-out cross-validation (LOOCV)
- Fold 하나에 하나의 샘플이 들어 있는 Stratified k-Fold 교차 검증
  - 즉, 각각의 반복에서 테스트 데이터에 하나의 샘플만 존재
  - 데이터셋이 클 때 시간이 매우 오래 걸림
  - 작은 데이터셋에 대해서는 일반적인 상황에 대한 거의 확실한 score 값을 얻을 수 있음.

In [14]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(logreg, iris.data, iris.target, cv=loo)
print("Number of cv iterations: ", len(scores))
print("Mean accuracy: {:.2f}".format(scores.mean()))

('Number of cv iterations: ', 150)
Mean accuracy: 0.95


#### Shuffle-split cross-validation
- 임의 분할 교차 검증
  - model_selection.SuffleSplit(n_splits=10, test_size='default') or model_selection.StratifiedSuffleSplit(n_splits=10, test_size='default')
    - n_splits: 10
      - 분할의 개수
    - test_size 만큼의 테스트 셋트를 만들도록 분할
      - test_size의 기본값: 0.1
  - 보통 test_size 값만 설정하며, 추가적으로 train_size 도 설정 가능
    - 이런 경우 전체 데이터 집합 중 일부만 훈련과 테스트에 사용할 수 있음
    - 대규모 데이터에 유용
  - test_size, train_size
    - 정수: 데이터 포인트의 개수
    - 실수: 데이터 포인트 비율<br/><br/>

- 아래 그림 예제
  - 전체 데이터 셈플 개수: 10
  - train_size = 5
  - test_size = 2
  - n_splits = 4

In [15]:
mglearn.plots.plot_shuffle_split()

NameError: name 'mglearn' is not defined

In [16]:
from sklearn.model_selection import ShuffleSplit

shuffle_split = ShuffleSplit(n_splits=10, test_size=.5, train_size=.5)
scores = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)

print("Cross-validation scores:\n{}".format(scores))
print("Mean accuracy: {:.2f}".format(scores.mean()))


Cross-validation scores:
[ 0.78666667  0.93333333  0.94666667  0.93333333  0.93333333  0.97333333
  0.96        0.96        0.93333333  0.96      ]
Mean accuracy: 0.93


In [17]:
from sklearn.model_selection import StratifiedShuffleSplit

shuffle_split = StratifiedShuffleSplit(n_splits=10, test_size=.5, train_size=.5)
scores = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)

print("Cross-validation scores:\n{}".format(scores))
print("Mean accuracy: {:.2f}".format(scores.mean()))


Cross-validation scores:
[ 0.92        0.97333333  0.96        0.92        0.97333333  0.94666667
  0.94666667  0.94666667  0.97333333  0.92      ]
Mean accuracy: 0.95


##### Cross-validation with groups
- 임의의 그룹에 속한 데이터 전체를 훈련 집합 또는 테스트 집합에 넣을 때 사용
- 테스트 데이터가 때때로 완전히 새로운 데이터가 되어야 할 필요 있음
- model_selection.GroupKFold
  - 그룹핑을 통하여 훈련 데이터 셋트와 테스트 데이터 셋트를 완벽히 분리하기 위해 사용
  - group 배열
    - 각 데이터 포인트 별로 그룹 index 지정 필요
    - 배열 내에 index 지정을 통해 훈련 데이터와 테스트 데이터를 랜덤하게 구성할 때 분리되지 말아야 할 그룹을 지정
    - 타깃 레이블과 혼동하면 안됨
- 더 나은 방법
  - 이 방법대신 model_selection.train_test_split을 통해 처음 부터 테스트 데이터를 미리 분리하는 것이 더 좋음.

In [18]:
mglearn.plots.plot_group_kfold()

NameError: name 'mglearn' is not defined

In [19]:
from sklearn.model_selection import GroupKFold

# create synthetic dataset
X, y = make_blobs(n_samples=12, random_state=0)

# assume the first three samples belong to the same group,
# then the next four, etc
groups = [0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3]
scores = cross_val_score(logreg, X, y, groups, cv=GroupKFold(n_splits=3))
print("Cross-validation scores:\n{}".format(scores))
print("Mean accuracy: {:.2f}".format(scores.mean()))

Cross-validation scores:
[ 0.75        0.8         0.66666667]
Mean accuracy: 0.74
